# PRÁCTICA 1

Lucía Pérez González, Manuel Ramallo Blanco, Alexandre Lorenzo Martínez

## 1 - Preprocesado

### 1.1 - Eliminación de duplicados

In [6]:
# Abrir datasets
import pandas as pd

df_vino = pd.read_csv("data/train.csv")

# Eliminación de duplicados, ignorado quality
cols = df_vino.columns.drop('quality')
df_vino = df_vino.drop_duplicates(subset=cols)

### 1.2 - Binarización de la calidad

In [7]:
def clasificar_vino(valor): 
    if valor < 7: 
        return "baja calidad" 
    elif valor >= 7: 
        return "alta calidad" 
df_vino['calidad'] = df_vino['quality'].apply(clasificar_vino) 
df_vino = df_vino.drop(columns=['quality'])

### 1.3 - Gestión de valores atípicos

Eliminamos aquellas filas que no tienen valores para 4 columnas o más:

In [8]:
df_vino = df_vino[df_vino.isnull().sum(axis=1) < 4]

Aquellas filas que tengan 4 o más valores atípicos también serán descartadas. Para contar cuántos datos atípicos hay por fila, añadimos una nueva columna.

In [9]:
df_vino['num atipicos'] = 0

#### 1.3.1 - Fixed acidity

#### 1.3.2 - Volatile acidity

#### 1.3.3 - Citric acid

#### 1.3.4 - Residual sugar

#### 1.3.5 - Chlorides

#### 1.3.6 - Free sulfur dioxide

#### 1.3.7 - Total sulfur dioxide

#### 1.3.8 - Density

#### 1.3.8 - pH

#### 1.3.9 - Sulphates

#### 1.3.10 - Alcohol

#### 1.3.11 - Eliminación de filas con muchos datos atípicos

In [10]:
# Lista de columnas numéricas (todas menos 'calidad' y 'num atipicos')
cols_diana = [c for c in df_vino.columns if c not in ["calidad", "num atipicos"]]

print("\nColumnas numéricas de características:")
print(cols_diana)

# Deteccion de valores atipicos

# Diccionario para guardar atípicos: fila -> columnas
atipicos_por_fila = {}

for col in cols_diana:

    # Calculamos IQR y límites
    Q1 = df_vino[col].quantile(0.25)
    Q3 = df_vino[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 3 * IQR
    upper = Q3 + 3 * IQR
    
    # Declaramos condición de atípico y registramos sus posiciones
    cond_iqr = (df_vino[col] < lower) | (df_vino[col] > upper)    #OLLO! Nico recomendara ver os casos 1 a 1 
    cond_neg = df_vino[col] < 0
    cond_atipico = cond_iqr | cond_neg

    for idx in df_vino[cond_atipico].index:
        if idx not in atipicos_por_fila:
            atipicos_por_fila[idx] = []
        atipicos_por_fila[idx].append(col)

    # Incrementamos el contador de atípicos por fila
    df_vino["num atipicos"] += cond_atipico.astype(int)

print("\nNúmero de filas con 0, 1, 2, ... atípicos (IQR + negativos):")
print(df_vino["num atipicos"].value_counts().sort_index())



Columnas numéricas de características:
['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']

Número de filas con 0, 1, 2, ... atípicos (IQR + negativos):
num atipicos
0    3188
1     493
2      43
3       2
4       1
Name: count, dtype: int64


In [11]:
df_vino = df_vino[df_vino['num atipicos'] < 4]